In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, mean, udf, lit, current_timestamp, unix_timestamp, array_contains
#from pyspark.sql.types import IntegerType, FloatType

import pandas as pd
import math
import mlflow
import os

In [2]:
# instantiate the Spark session

from pyspark.conf import SparkConf

sparkConf = SparkConf()
sparkConf.setAppName("My app").set("spark.jars", "/home/avani/UMass/Fall 2022/CS 532/Project/postgresql-42.5.0.jar")
sparkConf.set("spark.dynamicAllocation.enabled", "true")
sparkConf.set("spark.executor.cores", 8)
sparkConf.set("spark.dynamicAllocation.minExecutors","1")
sparkConf.set("spark.dynamicAllocation.maxExecutors","5000")
sparkConf.set("spark.executor.memory", "32g")
sparkConf.set("spark.ui.port","4050")
sparkConf.set("spark.memory.fraction", 0.7)

spark = SparkSession.builder.master('local[*]').config(conf=sparkConf).getOrCreate()

22/12/03 02:14:31 WARN Utils: Your hostname, avani-HP resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface wlo1)
22/12/03 02:14:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/03 02:14:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
tables_list = ['movies', 'ratings', 'tags', 'links', 'genome_tags', 'genome_scores']

dataframeList = {} #defaultdict(None)

for table in tables_list:

    if table == 'ratings':
        dataframeList[table] = spark.read.format("jdbc"). \
                                options(
                                 url = 'jdbc:postgresql://localhost:5432/movielens_dataset', # using jdbc:postgresql://<host>:<port>/<database>
                                 dbtable = table,
                                 user = 'postgres',
                                 password = 'postgres',
                                 driver = 'org.postgresql.Driver',
                                 fetchSize = 1000,
                                 partitionColumn = "userId",
                                 lowerBound = 1,
                                 upperBound = 283228,
                                 numPartitions = 32).\
                                load()
    else:
        dataframeList[table] = spark.read.format("jdbc"). \
                                options(
                                 url = 'jdbc:postgresql://localhost:5432/movielens_dataset', # using jdbc:postgresql://<host>:<port>/<database>
                                 dbtable = table,
                                 user = 'postgres',
                                 password = 'postgres',
                                 driver = 'org.postgresql.Driver').\
                                load()

In [4]:
for key, value in dataframeList.items():
    print(key + " table")
    print(value.printSchema())

movies table
root
 |-- movieid: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

None
ratings table
root
 |-- userid: integer (nullable = true)
 |-- movieid: integer (nullable = true)
 |-- rating: decimal(38,18) (nullable = true)
 |-- timestamp: integer (nullable = true)

None
tags table
root
 |-- userid: integer (nullable = true)
 |-- movieid: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)

None
links table
root
 |-- movieid: integer (nullable = true)
 |-- imdbid: integer (nullable = true)
 |-- tmdbid: integer (nullable = true)

None
genome_tags table
root
 |-- tagid: integer (nullable = true)
 |-- tag: string (nullable = true)

None
genome_scores table
root
 |-- movieid: integer (nullable = true)
 |-- tagid: integer (nullable = true)
 |-- relevance: decimal(38,18) (nullable = true)

None


In [5]:
movies_df = dataframeList['movies']
ratings_df = dataframeList['ratings']
links_df = dataframeList['links']
tags_df = dataframeList['tags']

### Show the dataframes and make the lifetime of dataframes sames as spark session

In [6]:
movies_df.show(20)
movies_df.createOrReplaceTempView("movies_df")

+-------+--------------------+--------------------+
|movieid|               title|              genres|
+-------+--------------------+--------------------+
|    346|     Backbeat (1993)|       Drama|Musical|
|    454|    Firm, The (1993)|      Drama|Thriller|
|    564|      Chasers (1994)|              Comedy|
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|   

In [7]:
ratings_df.show(20)
ratings_df.createOrReplaceTempView("ratings_df")

+------+-------+--------------------+----------+
|userid|movieid|              rating| timestamp|
+------+-------+--------------------+----------+
|   248|   1391|2.000000000000000000|1511187399|
|    73|   1230|4.500000000000000000|1437673250|
|    73|   1235|4.000000000000000000|1437671937|
|    73|   1237|4.000000000000000000|1437673718|
|    73|   1240|4.000000000000000000|1437836138|
|    73|   1244|5.000000000000000000|1437673251|
|    73|   1245|3.500000000000000000|1437673106|
|    73|   1246|4.000000000000000000|1437836025|
|    73|   1247|4.500000000000000000|1437673759|
|    73|   1248|4.000000000000000000|1437673640|
|    73|   1250|4.500000000000000000|1437671809|
|    73|   1252|4.000000000000000000|1437671836|
|    73|   1254|4.500000000000000000|1437671823|
|    73|   1258|4.500000000000000000|1437671985|
|    73|   1259|4.500000000000000000|1438015737|
|    73|   1265|3.500000000000000000|1437720392|
|    73|   1270|4.000000000000000000|1437836189|
|    73|   1273|4.00

In [8]:
links_df.show(20)
links_df.createOrReplaceTempView("links_df")

+-------+------+------+
|movieid|imdbid|tmdbid|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [9]:
tags_df.show(20)
tags_df.createOrReplaceTempView("tags_df")

+------+-------+--------------+----------+
|userid|movieid|           tag| timestamp|
+------+-------+--------------+----------+
|    14|    110|          epic|1443148538|
|    14|    110|      Medieval|1443148532|
|    14|    260|        sci-fi|1442169410|
|    14|    260|  space action|1442169421|
|    14|    318|  imdb top 250|1442615195|
|    14|    318|       justice|1442615192|
|    14|    480|     Dinosaurs|1443148563|
|    14|    593|psychothriller|1444014286|
|    14|   1682|    philosophy|1442615158|
|    14|   1682|  surveillance|1442615167|
|    14|   7458|          Epic|1443148675|
|    14|  95311|         Pixar|1443148494|
|    14| 117529|     dinosaurs|1443148646|
|    27|    260|classic sci-fi|1440448113|
|    27|    260| Harrison Ford|1440448123|
|    27|    260|      must see|1440448132|
|    27|    260|        sci-fi|1440448094|
|    42|  37733| disappointing|1264106059|
|    42|  37733|     overrated|1264106052|
|    42|  37733|        stupid|1264106067|
+------+---

### Registering the dataframes to spark 

In [10]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

/home/avani/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### Analyse the data

In [11]:
minRating_1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
minRating_2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()

print('Minimum number of ratings per user: {}'.format(minRating_1))
print('Minimum number of ratings per movie: {}'.format(minRating_2))

Minimum number of ratings per user: 1
Minimum number of ratings per movie: 1


In [12]:
_rating1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
_total = ratings_df.select('movieId').distinct().count()

print('movies are rated by only one user: {} out of {} '.format(_rating1, _total))

movies are rated by only one user: 10155 out of 53889 


In [13]:
# number of distinct users
num_users = spark.sql("SELECT count (distinct userID) as num_users FROM ratings")
ratings_df.select("userId").distinct().count()

283228

In [14]:
# number of movies
num_movies = spark.sql("SELECT count (distinct movieID) as num_movies FROM movies")
print(movies_df.select('movieID').distinct().count())

58098


In [15]:
rated_by_users = ratings_df.select('movieID').distinct().count()
print('Total Number of movies rated by users:', rated_by_users)

Total Number of movies rated by users: 53889


In [16]:
# movie genres
spark.sql("SELECT DISTINCT(genres) FROM movies LIMIT 10").show()

+--------------------+
|              genres|
+--------------------+
|Comedy|Horror|Thr...|
|Adventure|Sci-Fi|...|
|Action|Adventure|...|
| Action|Drama|Horror|
|Comedy|Drama|Horr...|
|Action|Animation|...|
|Fantasy|Musical|M...|
|Adventure|Mystery...|
|Children|Comedy|D...|
|Action|Adventure|...|
+--------------------+



In [17]:
extract_genres = udf(lambda x: x.split("|"), ArrayType(StringType()))
movies_df_clean = movies_df.select("movieId", "title", extract_genres("genres").alias("genres"))

movies_df_clean.createOrReplaceTempView("movies_df_clean")

display (spark.sql("SELECT * FROM movies_df_clean limit 5"))

DataFrame[movieId: int, title: string, genres: array<string>]

In [18]:
movies_df_clean.show(5)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|    346| Backbeat (1993)|    [Drama, Musical]|
|    454|Firm, The (1993)|   [Drama, Thriller]|
|    564|  Chasers (1994)|            [Comedy]|
|      1|Toy Story (1995)|[Adventure, Anima...|
|      2|  Jumanji (1995)|[Adventure, Child...|
+-------+----------------+--------------------+
only showing top 5 rows



In [19]:
# All movie categories
genres_result = list(set(movies_df_clean.select('genres').rdd.flatMap(tuple).flatMap(tuple).collect()))
genres_result

['Mystery',
 'Adventure',
 'Animation',
 'Film-Noir',
 'Horror',
 'Drama',
 'Thriller',
 'Action',
 'Documentary',
 'Fantasy',
 'Western',
 'Musical',
 'Comedy',
 'Children',
 'Sci-Fi',
 'War',
 'Crime',
 'IMAX',
 '(no genres listed)',
 'Romance']

In [20]:
movie_pdf = movies_df.toPandas()
list_of_movie = list(movie_pdf['title'])

## Recommender

In [21]:
# Data type convert
movie_ratings = ratings_df.drop('timestamp')

movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [22]:
movie_ratings.show(50)
movie_ratings.createOrReplaceTempView("movie_ratings")
display (spark.sql("SELECT * FROM movie_ratings limit 10"))

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   120|   1199|   4.0|
|   120|   1204|   3.5|
|   120|   1215|   3.0|
|   120|   1233|   4.0|
|   120|   1544|   4.5|
|   120|   2001|   3.0|
|   120|   2012|   3.0|
|   120|   2321|   3.5|
|   120|   3408|   4.0|
|   120|   7320|   3.0|
|   121|      1|   4.0|
|   121|     10|   4.0|
|   121|     11|   4.0|
|   121|     20|   4.0|
|   121|     31|   3.0|
|   121|     42|   4.0|
|   121|     48|   3.0|
|   121|     68|   4.0|
|   121|     95|   5.0|
|   121|    105|   3.0|
|   121|    122|   5.0|
|   121|    140|   4.0|
|   121|    145|   4.0|
|   121|    150|   5.0|
|   121|    151|   5.0|
|   121|    153|   4.0|
|   121|    160|   5.0|
|   121|    161|   4.0|
|   121|    165|   4.0|
|   121|    168|   5.0|
|   121|    172|   3.0|
|   121|    173|   3.0|
|   121|    174|   2.0|
|   121|    185|   3.0|
|   121|    186|   3.0|
|   121|    193|   2.0|
|   121|    204|   4.0|
|   121|    207|   5.0|
|   121|    208|

DataFrame[userId: int, movieId: int, rating: float]

In [23]:
movie_rating_sample = movie_ratings.sample(False, 1/500)
movie_rating_sample.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     4|   3717|   4.5|
|     4|   7348|   4.5|
|     5|   2485|   3.0|
|    43|   1942|   4.0|
|    43|   2997|   4.5|
|    43|   5349|   3.5|
|    43|   6003|   4.5|
|    45|     39|   1.0|
|    48|   3160|   5.0|
|    54|    141|   4.0|
|    71|   6016|   4.0|
|    71|  25771|   4.5|
|    73|   1193|   4.0|
|    73|   3275|   3.0|
|    73|   5746|   4.0|
|    79|   1586|   4.0|
|    79|   2716|   5.0|
|    81|  34319|   3.0|
|    81|  40946|   0.5|
|   100|   2268|   4.0|
+------+-------+------+
only showing top 20 rows

